In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import molsysmt as msm
import openexplorer as oe
import numpy as np
from simtk import unit
from simtk.openmm import app
import matplotlib.pyplot as plt

# MonteCarlo

In [ ]:
modeller = msm.convert('alanine_dipeptide.pdb', to_form='openmm.Modeller')

topology = modeller.topology
positions = modeller.positions

forcefield = app.ForceField('amber10.xml', 'amber10_obc.xml')
system = forcefield.createSystem(topology, constraints=app.HBonds, nonbondedMethod=app.NoCutoff)

explorer = oe.Explorer(topology, system, platform='CUDA')

In [ ]:
explorer.set_coordinates(positions)

In [ ]:
explorer.quench()

In [ ]:
movement = oe.movements.RandomDihedralMaxShifts(stepsize=45*unit.degrees)

In [ ]:
reporter = oe.reporters.OpenExplorerReporter(1, step=True, coordinates=True, boxVectors=False,
                                             potentialEnergy=True)

In [ ]:
exploration = oe.exploration_campaign.MonteCarlo(explorer, movement, temperature=300.0*unit.kelvin,
                                                 reporter=reporter)

In [ ]:
exploration.run(1000)

In [ ]:
exploration.n_tries

In [ ]:
exploration.n_acceptances

In [ ]:
plt.plot(exploration.reporter.step, exploration.reporter.potential_energy._value)
plt.show()

In [ ]:
view = msm.view(exploration.reporter)
view.clear()
view.add_licorice()
view

In [ ]:
phi_chains, psi_chains, phi_angles, psi_angles = msm.ramachandran_angles(exploration.reporter)

In [ ]:
import seaborn as sns

ax = sns.kdeplot(phi_angles[:,0], psi_angles[:,0], shade=True)
ax.set_xlim(-180.0,180.0)
ax.set_ylim(-180.0,180.0)
ax.set_xticks([-180.0, -90.0, 0.0, 90.0, 180.0])
ax.set_yticks([-180.0, -90.0, 0.0, 90.0, 180.0])
ax.set_xlabel('$\phi_1$')
ax.set_ylabel('$\psi_1$')
ax.set_aspect('equal')

In [ ]:
msm.covalent_dihedral_quartets(exploration.reporter, dihedral_angle='phi')

In [ ]:
msm.get_form(exploration.reporter.topology)

In [ ]:
exploration.reset(temperature=900.0*unit.kelvin)

In [ ]:
exploration.run(1000)

In [ ]:
exploration.n_tries

In [ ]:
exploration.n_acceptances